The quote from Ritzwoller & Romano 2021:
> ...multiple testing procedures reveal that one shooter exhibits a shooting pattern significantly inconsistent with randomness – supplying strong evidence that basketball shooting is not random for all shooters all of the time. However, we find that the evidence against randomness in this experiment is limited to this shooter

link: https://www.restud.com/paper/uncertainty-in-the-hot-hand-fallacy-detecting-streaky-alternatives-to-random-bernoulli-sequences/

# Skip to (3) for results

## 1. Three Stata rclass commands:  (skip over this for results)
1. Generating random data
2. Running a fixed effect regression coefficient (biased)
3. Running a biased-corrected test on the regression coefficient

In [1]:
{


cap program drop anGVT_fe
program define anGVT_fe, rclass
	syntax [,  k(integer 3) command(string) ]
	*This code regenerates GVT's shot outcomes, randomly 
	*Then runs a fixed effect regression (biased), and returns the coefficient estimate 
	*hitting k or more in row
	
	clear
	use gilovichshooters.dta
	
	*Flexible, to add any command here:
	`command'
	
	*randomize the shot outcomes
	by sid: egen fg=mean(make)
	gen rand1=runiform()
	replace make = (rand1<=fg) 
	

	
	*Build streak condition statement
	local make="make[_n-1]==1"
	local miss="make[_n-1]==0"
	forvalues i=2(1)`k'{
		local make="`make' & make[_n-`i']==1"
		local miss="`miss' & make[_n-`i']==0"
	}
	
	*Generate the regressor:
	gen dMade`k'ormore=`make' if period>`k'
	gen dMissed`k'ormore=`miss' if period>`k'
	
	*fixed effect regression 
	reg make dMade`k'ormore i.sid if dMade`k'ormore==1| dMissed`k'ormore==1
	
	*return the coefficent
	return scalar diff=el(r(table),1,1)
end


cap program drop ttest_beta
program define ttest_beta, rclass
	syntax [, k(integer 3) sim_data(string) command(string) ]
	*Is the hot hand in GVT robust 
quietly{
	clear
	*Load
	use gilovichshooters.dta
	do genregressionvars_v2.do  //generate the variables that are used in the regression
	genregressionvars	
	`command'
	
	inspect sid
	if r(N_unique)> 1{
		xtset sid 
		noi: xtreg make dMade`k'ormore if dMade`k'ormore==1| dMissed`k'ormore==1, fe
	}
	else{
		noi: reg make dMade`k'ormore if dMade`k'ormore==1| dMissed`k'ormore==1
	}
	clear
	use `sim_data'
	summarize, meanonly
	local bias =  r(mean) 
	
	*Alternative Hypothsis 1: Hot or Cold Hand
	noi: test dMade`k'ormore=`bias'
	
	return scalar hot_hand_effect = _b[dMade`k'ormore] - `bias'		
	*Alternative Hypothesis 2: Hot hand (one-sided test, justified by priors?)
		local tstat=sign(_b[dMade`k'ormore])*sqrt(r(F))
		
		return scalar tstat = `tstat'
		return scalar pvalue_right_tail =  ttail(r(df_r) ,`tstat')	
}
end

}

## 2 Simulate coefficient estimates to calculate bias (in the context of fe regression)
This will take 5-10 min

In [3]:
*For sid 109 only, k=3,4
simulate diff=r(diff),reps(10000) saving(MS_Bias_only109_k3.dta): anGVT_fe,  k(3) command("keep if sid == 109") 
simulate diff=r(diff),reps(10000) saving(MS_Bias_only109_k4.dta): anGVT_fe,  k(4) command("keep if sid == 109") 

*For GVT without sid 109, k=3,4
simulate diff=r(diff),reps(10000) saving(MS_Bias_without109_k3.dta): anGVT_fe,  k(3) command("drop if sid == 109")
simulate diff=r(diff),reps(10000) saving(MS_Bias_without109_k4.dta): anGVT_fe,  k(4) command("drop if sid == 109")



      command:  anGVT_fe, k(3) command("keep if sid == 109")
         diff:  r(diff)

Simulations (10000)
----+--- 1 ---+--- 2 ---+--- 3 ---+--- 4 ---+--- 5 
..................................................    50
..................................................   100
..................................................   150
..................................................   200
..................................................   250
..................................................   300
..................................................   350
..................................................   400
..................................................   450
..................................................   500
..................................................   550
..................................................   600
..................................................   650
..................................................   700
..........................................

..................................................  7100
..................................................  7150
..................................................  7200
..................................................  7250
..................................................  7300
..................................................  7350
..................................................  7400
..................................................  7450
..................................................  7500
..................................................  7550
..................................................  7600
..................................................  7650
..................................................  7700
..................................................  7750
..................................................  7800
..................................................  7850
..................................................  7900
...............................

..................................................  4150
..................................................  4200
..................................................  4250
..................................................  4300
.........x........................................  4350
..................................................  4400
..................................................  4450
..................................................  4500
..........x.......................................  4550
..................................................  4600
..................................................  4650
..................................................  4700
..................................................  4750
..........................................x.......  4800
..................................................  4850
..................................................  4900
..................................................  4950
...............................

..................................................  1200
..................................................  1250
..................................................  1300
..................................................  1350
..................................................  1400
..................................................  1450
..................................................  1500
..................................................  1550
..................................................  1600
..................................................  1650
..................................................  1700
..................................................  1750
..................................................  1800
..................................................  1850
..................................................  1900
..................................................  1950
..................................................  2000
...............................

..................................................  8400
..................................................  8450
..................................................  8500
..................................................  8550
..................................................  8600
..................................................  8650
..................................................  8700
..................................................  8750
..................................................  8800
..................................................  8850
..................................................  8900
..................................................  8950
..................................................  9000
..................................................  9050
..................................................  9100
..................................................  9150
..................................................  9200
...............................

..................................................  5450
..................................................  5500
..................................................  5550
..................................................  5600
..................................................  5650
..................................................  5700
..................................................  5750
..................................................  5800
..................................................  5850
..................................................  5900
..................................................  5950
..................................................  6000
..................................................  6050
..................................................  6100
..................................................  6150
..................................................  6200
..................................................  6250
...............................

## 3.  How much of a hot hand does shooter 109 have?  A lot

In [4]:
ttest_beta, k(3) sim_data(MS_Bias_only109_k3.dta) command("keep if sid ==109")

disp "++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++"
disp "Bias Corrected Hot hand Effect = " r(hot_hand_effect)
disp "Pvalue of Test with hot hand alternative = "  r(pvalue_right_tail)



      Source |       SS           df       MS      Number of obs   =        50
-------------+----------------------------------   F(1, 48)        =     15.44
       Model |  2.80333333         1  2.80333333   Prob > F        =    0.0003
    Residual |  8.71666667        48  .181597222   R-squared       =    0.2433
-------------+----------------------------------   Adj R-squared   =    0.2276
       Total |       11.52        49  .235102041   Root MSE        =    .42614

------------------------------------------------------------------------------
        make |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
dMade3ormore |   .4833333   .1230167     3.93   0.000     .2359917    .7306749
       _cons |        .35   .0952883     3.67   0.001       .15841      .54159
------------------------------------------------------------------------------

 ( 1)  dMade3ormore = -.0819733

       F(  1,  

**Robust to stricter definition of a streak? Length 4 or longer:**

In [5]:
ttest_beta, k(4) sim_data(MS_Bias_only109_k4.dta) command("keep if sid ==109")

disp "++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++"
disp "Bias Corrected Hot hand Effect = " r(hot_hand_effect)
disp "Pvalue of Test with hot hand alternative = "  r(pvalue_right_tail)



      Source |       SS           df       MS      Number of obs   =        38
-------------+----------------------------------   F(1, 36)        =      7.51
       Model |   1.4757085         1   1.4757085   Prob > F        =    0.0095
    Residual |  7.07692308        36  .196581197   R-squared       =    0.1725
-------------+----------------------------------   Adj R-squared   =    0.1496
       Total |  8.55263158        37  .231152205   Root MSE        =    .44337

------------------------------------------------------------------------------
        make |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
dMade4ormore |   .4153846   .1516076     2.74   0.010     .1079101    .7228592
       _cons |   .3846154     .12297     3.13   0.003     .1352206    .6340102
------------------------------------------------------------------------------

 ( 1)  dMade4ormore = -.1605503

       F(  1,  

## 4.  Is GVT significant without shooter 109 have?  Yes*, hot hand effect = 10.2pp (SE 4.6)
*Yes for hot hand alternative, borderline for cold or hot hand alternative

In [9]:
disp "Unadjusted (biased) Estimate:"
disp "---------------------------"
ttest_beta, k(3) sim_data(MS_Bias_without109_k3.dta) command("drop if sid ==109")

disp "++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++"
disp "Bias Corrected Hot hand Effect = " r(hot_hand_effect)
disp "Pvalue of Test with hot hand alternative = "  r(pvalue_right_tail)


Unadjusted (biased) Estimate:

---------------------------


Fixed-effects (within) regression               Number of obs     =        663
Group variable: sid                             Number of groups  =         25

R-sq:                                           Obs per group:
     within  = 0.0027                                         min =         13
     between = 0.5772                                         avg =       26.5
     overall = 0.0190                                         max =         38

                                                F(1,637)          =       1.71
corr(u_i, Xb)  = 0.3871                         Prob > F          =     0.1912

------------------------------------------------------------------------------
        make |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
dMade3ormore |    .059581   .0455326     1.31   0.191    -.0298312    .1489932
   

**Robust to stricter definition of a streak? Length 4 or longer: hot hand effect = 12.4pp (SE 6.4)**

In [12]:
ttest_beta, k(4) sim_data(MS_Bias_without109_k4.dta) command("drop if sid ==109")

disp "++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++"
disp "Bias Corrected Hot hand Effect = " r(hot_hand_effect)
disp "Pvalue of Test with hot hand alternative = "  r(pvalue_right_tail)



Fixed-effects (within) regression               Number of obs     =        375
Group variable: sid                             Number of groups  =         25

R-sq:                                           Obs per group:
     within  = 0.0012                                         min =          5
     between = 0.4804                                         avg =       15.0
     overall = 0.0268                                         max =         25

                                                F(1,349)          =       0.44
corr(u_i, Xb)  = 0.4644                         Prob > F          =     0.5093

------------------------------------------------------------------------------
        make |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
dMade4ormore |     .04332   .0655818     0.66   0.509    -.0856653    .1723052
       _cons |   .4171076   .0368046    11.33   0.000      .344